In [5]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate




from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [6]:
siglimit = 10. # sigma
read_noise = 3. # Read noise for two frames

duet = config.Telescope()
duet.info()
print()
neff = duet.neff
print('Number of effective bgd pixels: {}'.format(neff))



# point_drift = 1*u.arcsec # To be added to the PSF in quadrature
# point_jitter = 5*u.arcsec


# # Transmission efficiency
# trans_eff = (0.975)**8 # from Jim. 

# print('Transmission efficiency: {}'.format(trans_eff))

# # Load telescope info
# config=11
# diameter, qe, psf_size, pixel_size, efficiency = load_telescope_parameters(config)

# area = np.pi * (diameter*0.5)**2
# area_onaxis = np.pi * (0.5*24.2*u.cm)**2
# efficiency = area_onaxis / area

# print('Effective area (used) {}'.format(area*efficiency))

# # Add in fuzz...
# psf_size = (psf_size**2 + point_drift**2 + point_jitter**2)**0.5


# print('Effective PSF size {}'.format(psf_size))
# # Get the number of effective background pixels
# print()


-----
DUET Telescope State:
Physical Entrance Pupil: 26.0 cm
Effective EPD: 24.2 cm
Effective Area: 459.9605804120816 cm2
Transmission Efficiency: 0.8166518036622619

Pixel size: 6.4 arcsec
PSF FWHM: 5.0 arcsec
Pointing jitter: 5.0 arcsec
Effective PSF FWHM: 7.0710678118654755 arcsec
N_eff: 3.3581908798935647

Band 1: {'eff_wave': <Quantity 202.56878682 nm>, 'eff_width': <Quantity 53.32814342 nm>}
Bandpass 1: [175.90471511 229.23285853] nm
Band 2: {'eff_wave': <Quantity 281.7531854 nm>, 'eff_width': <Quantity 68.16239088 nm>}
Bandpass 2: [247.67198996 315.83438085] nm


Dark current: 0.0115 ph / s
Read noise (RMS per read): 7
-----

Number of effective bgd pixels: 3.3581908798935647


In [7]:
# Band one
light=True

center = 208
width = 53
bandone=[center - 0.5*width, center+0.5*width]*u.nm
bandone = duet.bandpass1

print('For 10e3 K source')
for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, light=light)
    band1_rate = duet.trans_eff  * duet.eff_area * band1_fluence

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))


        break
print()
print('For 20e3 K source')


for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
    band1_rate = duet.trans_eff  * duet.eff_area * band1_fluence

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))
        break


For 10e3 K source
Band 1 Photometric Zero Point (Swift UVW2): 20.950000000000042
Band 1 Photometric Zero Point: 20.9558022146953 mag(AB)

For 20e3 K source
Band 1 Photometric Zero Point (Swift UVW2): 20.75000000000004
Band 1 Photometric Zero Point: 20.751702187075658 mag(AB)


In [8]:
light=True

center = 284
width = 68
bandtwo=[center - 0.5*width, center+0.5*width]*u.nm
bandtwo = duet.bandpass2

for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, light=light)
    band2_rate = duet.trans_eff  * duet.eff_area * band2_fluence

    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break
        
print()       
for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
    band2_rate = duet.trans_eff  * duet.eff_area * band2_fluence

    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break

Band 2 Photometric Zero Point: 21.750000000000053
Band 2 Photometric Zero Point: 20.657012356483918 mag(AB)

Band 2 Photometric Zero Point: 20.650000000000038
Band 2 Photometric Zero Point: 20.572274537652 mag(AB)


In [18]:
light=True
siglimit=10


exp1 = []
sm = []

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
# Compute sky background
# bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
#                               diameter=diameter,
#                               pixel_size=pixel_size, real_red=True, light=light)
#bgd_band1 = duet.* bgd_ph_rate
print('Sky background rate per pixel band1: {}'.format(bgd_band1))

# bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
#                               diameter=diameter,
#                               qe_band=2,
#                               pixel_size=pixel_size, real_red=True, light=light)
#bgd_band2 = trans_eff  * efficiency * bgd_ph_rate
print('Sky background rate per pixel band2: {}'.format(bgd_band2))


bandone = duet.bandpass1
bandtwo = duet.bandpass2

# center = 208
# width = 53
# bandone=[center - 0.5*width, center+0.5*width]*u.nm

# center = 284
# width = 68
# bandtwo=[center - 0.5*width, center+0.5*width]*u.nm



print()
for siglimit in [5]:#, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900, 1200]):
        for swiftmag in np.arange(20, 24, 0.05):
            band1_fluence, foo = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, light=light)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            exposure1 = calc_exposure(siglimit, band1_rate, bgd_band1,read_noise*(nframes+1)**0.5, neff)
            if (exposure1.value > explimit):
                bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
                print('Band 1 {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag1))
                break

print()

for siglimit in [5]:#, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900, 1200]):
        for swiftmag in np.arange(20, 24, 0.05):
            foo, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, light=light)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            exposure2 = calc_exposure(siglimit, band2_rate, bgd_band2,read_noise*(nframes+1)**0.5, neff)
            if (exposure2.value > explimit):
                foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
                print('Band 2 {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag2))
                break

Sky background rate per pixel band1: 0.030369732491096913 ph / s
Sky background rate per pixel band2: 0.2478588509265617 ph / s


Band 1 300-s 5-σ magnitude limit: 22.650470091092423 mag(AB)
Band 1 600-s 5-σ magnitude limit: 23.15047009109243 mag(AB)
Band 1 900-s 5-σ magnitude limit: 23.40047009109243 mag(AB)
Band 1 1200-s 5-σ magnitude limit: 23.600470091092433 mag(AB)


Band 2 300-s 5-σ magnitude limit: 21.83825736350797 mag(AB)
Band 2 600-s 5-σ magnitude limit: 22.238257363507977 mag(AB)
Band 2 900-s 5-σ magnitude limit: 22.488257363507977 mag(AB)
Band 2 1200-s 5-σ magnitude limit: 22.68825736350798 mag(AB)


In [15]:
light=True
siglimit=10


exp1 = []
sm = []

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
# Compute sky background
# bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
#                               diameter=diameter,
#                               pixel_size=pixel_size, real_red=True, light=light)
#bgd_band1 = duet.* bgd_ph_rate
print('Sky background rate per pixel band1: {}'.format(bgd_band1))

# bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
#                               diameter=diameter,
#                               qe_band=2,
#                               pixel_size=pixel_size, real_red=True, light=light)
#bgd_band2 = trans_eff  * efficiency * bgd_ph_rate
print('Sky background rate per pixel band2: {}'.format(bgd_band2))


bandone = duet.bandpass1
bandtwo = duet.bandpass2

# center = 208
# width = 53
# bandone=[center - 0.5*width, center+0.5*width]*u.nm

# center = 284
# width = 68
# bandtwo=[center - 0.5*width, center+0.5*width]*u.nm

bgd_rate = bgd_band1 + bgd_band2


print()
for siglimit in [5]:#, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900, 1200, 1500]):
        for swiftmag in np.arange(20, 24, 0.05):
            band1_fluence, foo = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, light=light)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            foo, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, light=light)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            tot_rate= band1_rate + band2_rate
            exposure1 = calc_exposure(siglimit, tot_rate, bgd_rate,read_noise*(nframes*2+1)**0.5, neff)
            if (exposure1.value > explimit):
                bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
                print('Stacked {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag1))
                break

print()


Sky background rate per pixel band1: 0.030369732491096913 ph / s
Sky background rate per pixel band2: 0.2478588509265617 ph / s


Stacked 300-s 5-σ magnitude limit: 22.850470091092426 mag(AB)
Stacked 600-s 5-σ magnitude limit: 23.25047009109243 mag(AB)
Stacked 900-s 5-σ magnitude limit: 23.450470091092434 mag(AB)
Stacked 1200-s 5-σ magnitude limit: 23.650470091092437 mag(AB)
Stacked 1500-s 5-σ magnitude limit: 23.75047009109244 mag(AB)

